# Mount To Drive & Import Libraries

In [11]:
# CHANGE THIS TO YOUR DIRECTORY
embedding_dir = '/content/drive/MyDrive/11785_Project/Programming/embeddings/batch_embedding_test'
# CHANGE THIS TO YOUR EMBEDDING WIDTH
embedding_width = 768

import os
from google.colab import drive
drive.mount("/content/drive")

embedding_files = os.listdir(embedding_dir)
embedding_files = [f for f in embedding_files if f.endswith('.json')]

print(f'Running downstream tasks on the following embedding files:')
for f in embedding_files:
    print(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Running downstream tasks on the following embedding files:
pre_trained_CLS_distilroberta-base_0_1651152434_last_layer_distil_roberta.json
pre_trained_avg_distilroberta-base_0_1651152434_last_layer_distil_roberta.json
pre_trained_avg_distilbert-base-uncased_0_1651112779_last_layer_distil_roberta.json


In [4]:
import pandas
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

!pip install sentence_transformers

from sentence_transformers import SentenceTransformer

     |████████████████████████████████| 79 kB 3.7 MB/s 
     |████████████████████████████████| 4.0 MB 29.3 MB/s 
     |████████████████████████████████| 1.2 MB 34.2 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 895 kB 40.0 MB/s 
     |████████████████████████████████| 596 kB 48.9 MB/s 
     |████████████████████████████████| 6.6 MB 36.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=af899bc1a2ba402b9a676ee694828d5fa65c3ee78670d5435f744e76f99aa0e6
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Phrasebank DataLoader

In [5]:
# For Downstream Data - Phrasebank
!mkdir downstream_data_zip
!mkdir downstream_data

!cp /content/drive/MyDrive/11785_Project/Programming/Data/phrasebank.zip downstream_data_zip

!unzip -q downstream_data_zip/*.zip -d downstream_data

In [6]:
from pandas.core.frame import DataFrame
class LibriSamplesPhrasebank(torch.utils.data.Dataset):
    def __init__(self, data:DataFrame, convert_label_to_int=True, return_labels=False, sentence_to_vec:dict=None):
        """
        data:DataFrame
            This is the data split from the all-data.csv in the Financial Phrase Bank
        
        convert_label_to_int:bool
            If this is true, then:
                 negative = -1
                 neutral  =  0
                 positive =  1
        
        sentence_to_vec:dict
            If this value is present, then dictionary of sentence to vector mappings.
        """
        # Simply reading in the csv
        self.X = data["sentence"].to_numpy()
        self.unresolved = set()
        # Convert the sentences to vectors if a dictionary is provided
        if sentence_to_vec != None:
            new_X = []
            for sentence in self.X:
                vector = sentence_to_vec.get((sentence,), None)
                if vector == None:
                    # self.unresolved.add(sentence)
                    # print("The dictionary contains no vector for the sentence: \n{}".format(sentence))
                    # continue
                    raise Exception("The dictionary contains no vector for the sentence: \n{}".format(sentence))
                    return
                new_X.append(vector)
            self.X = np.array(new_X)
        else:
            new_X = []
            default_model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
            for sentence in self.X:
                vector = default_model.encode(sentence) # numpy.array D=384
                new_X.append(torch.from_numpy(vector))
            self.X = np.array(new_X)

        # Check if we should be returning labels (Not necessary for generating the BERT Emedding)
        self.return_labels = return_labels
        if return_labels:
            self.Y = data["label"].to_numpy()
            # Probably will want to convert the data into numeric form for easier handling
            if convert_label_to_int:
                new_Y = np.ones(len(self.Y), dtype=np.int8)
                new_Y[self.Y == "negative"] = 0
                new_Y[self.Y == "positive"] = 2
                self.Y = torch.from_numpy(new_Y)

    def get_unresolved(self):
        return self.unresolved

    def __len__(self):
        """
        Get the size of the data.
        """
        return len(self.X)
    
    def __getitem__(self, ind):
        """
        if self.return_labels==True, then this will return the sentence and corresponding label,
        otherwise this will return only the sentence
        """
        if self.return_labels:
            return self.X[ind], self.Y[ind]
        else:
            return self.X[ind]

class LibriSamplesPhrasebankWordEmbedding(torch.utils.data.Dataset):
    def __init__(self, data:DataFrame, convert_label_to_int=True, return_labels=False, word_to_vec=None):
        """
        data:DataFrame
            This is the data split from the all-data.csv in the Financial Phrase Bank
        
        convert_label_to_int:bool
            If this is true, then:
                 negative = -1
                 neutral  =  0
                 positive =  1
        
        word_to_vec: lambda
            If this value is present, then dictionary of word to vector mappings.
        """
        # Simply reading in the csv
        self.X = data["sentence"].to_numpy()

        # Convert the sentences to vectors if a dictionary is provided
        if word_to_vec != None:
            new_X = []
            new_lenX = []
            for sentence in self.X:
                word_vec = []
                for word in sentence.split():
                    word = word.lower()
                    try:
                        vector = word_to_vec(word)
                        word_vec.append(vector)
                    except:
                        # print(f'Cannot embed: {word}')
                        pass

                    # if word in word_to_vec.keys():
                    #     vector = word_to_vec[word]
                    #     word_vec.append(vector)
                if len(word_vec) > 0:
                    new_X.append(np.array(word_vec))
                    new_lenX.append(len(word_vec))
                # else:
                #     print(f'could not recog any word in: {sentence}')
            self.X = np.array(new_X)

        # Check if we should be returning labels (Not necessary for generating the BERT Emedding)
        self.return_labels = return_labels
        if return_labels:
            self.Y = data["label"].to_numpy()
            # Probably will want to convert the data into numeric form for easier handling
            if convert_label_to_int:
                new_Y = np.ones(len(self.Y), dtype=np.int64)
                new_Y[self.Y == "negative"] = 0
                new_Y[self.Y == "positive"] = 2
                self.Y = new_Y

    def __len__(self):
        """
        Get the size of the data.
        """
        return len(self.X)
    
    def __getitem__(self, ind):
        """
        if self.return_labels==True, then this will return the sentence and corresponding label,
        otherwise this will return only the sentence
        """
        if self.return_labels:
            return torch.from_numpy(self.X[ind]), self.Y[ind]
        else:
            return torch.from_numpy(self.X[ind])
    
    def labeled_collate_fn(batch):
        batch_x = [x for x,y in batch]
        batch_y = [y for x,y in batch]

        batch_x_pad = pad_sequence(batch_x, batch_first=True)
        lengths_x = [len(x) for x in batch_x]

        return batch_x_pad, torch.tensor(lengths_x), torch.tensor(batch_y)
    
    def unlabeled_collate_fn(batch):
        batch_x = batch

        batch_x_pad = pad_sequence(batch_x, batch_first=True)
        lengths_x = [len(x) for x in batch_x]

        return batch_x_pad, torch.tensor(lengths_x)

# Modelling

In [7]:
csv_path = "/content/downstream_data/all-data.csv"
df = pandas.read_csv(csv_path, encoding="ISO-8859-1", names=["label", "sentence"])

df_train, df_val = train_test_split(df, test_size = 0.1, random_state = 10)
len(df_train), len(df_val)

(4361, 485)

# Sentence Embedding

In [8]:
class SentenceModel(nn.Module):
    def __init__(self, embedding_size: int, classes: int):
        super(SentenceModel, self).__init__()
        self.classification = nn.Linear(
            in_features=embedding_size,
            out_features=classes,
        )
        self.softmax = nn.Softmax(dim = 1)

    def forward(self, inputs_):
        x = self.classification(inputs_)
        x = self.softmax(x)
        return x

In [9]:
lr = 1e-3
epochs = 100
classes = 3

# sentence_to_vec = None
# embedding_width = 384
# sentence_to_vec = torch.load('/content/drive/MyDrive/11785_Project/Programming/embeddings/pre_trained_avg_distilbert-base-uncased_0_1651112779_last_layer_distil_roberta.json')
# embedding_width = 768
# sentence_to_vec = torch.load('/content/drive/MyDrive/11785_Project/Programming/embeddings/pre_trained_CLS_distilroberta-base_0_1651152434_last_layer_distil_roberta.json')
# embedding_width = 768
# sentence_to_vec = torch.load('/content/drive/MyDrive/11785_Project/Programming/embeddings/pre_trained_avg_distilroberta-base_0_1651152434_last_layer_distil_roberta.json')
# embedding_width = 768
# sentence_to_vec = torch.load('/content/drive/MyDrive/11785_Project/Programming/embeddings/avg_retrained_distilroberta-base_0_1651228176.json')
# embedding_width = 768
# sentence_to_vec = torch.load('/content/drive/MyDrive/11785_Project/Programming/embeddings/bert-base-uncased-baseline_0_1651128098_last_layer_distil_roberta.json')
# embedding_width = 768
# sentence_to_vec = torch.load('/content/drive/MyDrive/11785_Project/Programming/embeddings/bert-base-uncased-trained_0_1651200461_last_layer_distil_roberta.json')
# embedding_width = 768
# sentence_to_vec = torch.load('/content/drive/MyDrive/11785_Project/Programming/embeddings/distilbert-base-uncased_0_1651117834_distilbert-base-uncased_BASELINE.json')
# embedding_width = 768
# sentence_to_vec = torch.load('/content/drive/MyDrive/11785_Project/Programming/embeddings/distilbert-base-uncased_1_1651148962_distilbert-base-uncased.json')
# embedding_width = 768
# sentence_to_vec = torch.load('/content/drive/MyDrive/11785_Project/Programming/embeddings/distilbert-base-multilingual-cased_0_1651127952_last_layer_distil_bert-mult-cased-base.json')
# embedding_width = 768
# sentence_to_vec = torch.load('/content/drive/MyDrive/11785_Project/Programming/embeddings/distilbert-base-multilingual-cased_0_1651155335_last_layer_distil_bert-mult-cased.json')
# embedding_width = 768




In [10]:

def train_model(model):
    torch.cuda.empty_cache()
    model.train()
    total_loss = 0
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()

        x = x.cuda()
        y = y.type(torch.LongTensor)
        y = y.cuda()

        # Don't be surprised - we just wrap these two lines to make it work for FP16
        h = model(x)
        loss = criterion(h, y)

        # Update loss as we go
        total_loss += float(loss)

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))))
        
        # Another couple things you need for FP16. 
        loss.backward() # This is a replacement for loss.backward()
        optimizer.step() # This is a replacement for optimizer.step()
        batch_bar.update() # Update tqdm bar
    scheduler.step()
    batch_bar.close() # You need this to close the tqdm bar

    loss = float(total_loss / len(train_loader))
    print("Epoch {}/{}: Train Loss {:.04f}".format(
        epoch + 1,
        epochs,
        loss))

    return loss

def eval_model(model):
    torch.cuda.empty_cache()
    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')
    dists = []
    correct = 0.0
    total = 0.0

    for i, data in enumerate(val_loader):
        x, y = data
        x = x.cuda()
        y = y.type(torch.LongTensor)
        y = y.cuda()

        h = model(x)
        preds = torch.argmax(h, axis=1, keepdims=False)
        # print(f'preds: {preds}')
        # print(f'y: {y}')
        correct += (preds == y).float().sum()
        total += len(preds)
        batch_bar.update()
    
    batch_bar.close()

    score = correct / total
    print(f"Validation acc: {score}")
    return score

In [12]:

results = []

for embedding_file in embedding_files:

    sentence_to_vec = torch.load(f'{embedding_dir}/{embedding_file}')

    train_data = LibriSamplesPhrasebank(df_train, convert_label_to_int=True, return_labels=True, sentence_to_vec=sentence_to_vec)
    train_loader = DataLoader(train_data, batch_size=128)

    val_data = LibriSamplesPhrasebank(df_val, convert_label_to_int=True, return_labels=True, sentence_to_vec=sentence_to_vec)
    val_loader = DataLoader(val_data, batch_size=128)

    model = SentenceModel(embedding_width, classes).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr) # LR from https://arxiv.org/pdf/1903.10676.pdf
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs, eta_min=lr*0.05)

    for epoch in range(epochs):
        lr = float(optimizer.param_groups[0]['lr'])
        loss = train_model(model)
        score = eval_model(model)
    
    results.append((embedding_file, score))

with open(f'{embedding_dir}/results.csv', 'w') as f:
    for file, score in results:
        print(f'{file}: {score}')
        f.write(f'{file}, {score}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Epoch 1/100: Train Loss 0.9808


Validation acc: 0.6082474589347839


Epoch 2/100: Train Loss 0.9579


Validation acc: 0.6082474589347839


...

Validation acc: 0.6082474589347839


Epoch 99/100: Train Loss 1.0731


Validation acc: 0.6082474589347839


Epoch 100/100: Train Loss 1.0731


Validation acc: 0.6082474589347839
pre_trained_CLS_distilroberta-base_0_1651152434_last_layer_distil_roberta.json: 0.73195880651474
pre_trained_avg_distilroberta-base_0_1651152434_last_layer_distil_roberta.json: 0.6721649765968323
pre_trained_avg_distilbert-base-uncased_0_1651112779_last_layer_distil_roberta.json: 0.6082474589347839
